## Zadanie domowe: BBHE i DSIHE

W klasycznym wyrównywaniu histogramu HE  po wykonaniu operacji jasność obrazu ulega zmianie.
Dało się to zaobserwować podczas przeprowadzonych eksperymentów.
Jeśli nie to należy uruchomić skrypt z sekcji A i zwrócić na to uwagę.
Średnia jasność dąży do środkowego poziomu szarości.
Jest to wada i dlatego klasyczne HE ma ograniczone zastosowanie.

Powstało sporo metod, które eliminują to niekorzystne zjawisko.
Najprostsze z nich polegają na dekompozycji obrazu wejściowego na dwa podobrazy (wg. pewnego kryterium).
Następnie operacja HE wykonywana jest dla tych podobrazów.

Dwie znane z literatury metody to:
- Bi-Histogram Equalization
- DSIHE - Dualistic Sub-Image Histogram Equalization

W metodzie BBHE za kryterium podziału przyjmuje się średnią jasność w obrazie.
W DSIHE obraz dzieli się na dwa podobrazy o takiej samej liczbie pikseli (jaśniejszych i ciemniejszych).

W ramach zadania należy zaimplementować wybraną metodę: BBHE lub DSIHE (ew. obie).

1. Wczytaj obraz *jet.bmp* i wylicz jego histogram.
2. W kolejnym kroku należy wyznaczyć próg podziału obrazu na dwa podobrazy (*lm*).
3. Dla BBHE wyznacz średnią jasność obrazu. Dla DSIHE można wykorzystać histogram skumulowany.
Należy znaleźć poziom jasności który znajduje się "w połowie" histogramu skumulowanego.
W tym celu warto stworzyć tablicę, zawierającą moduł histogramu skumulowanego pomniejszonego o połowę liczby pikseli.
Następnie znaleźć minimum - `np.argmin`.
4. Dalej należy podzielić histogram oryginalnego obrazu na dwa histogramy *H1* i *H2*.
Dla każdego z nich wyliczyć histogram skumulowany ($C_1$ i $C_2$) i wykonać normalizację.
Normalizacja polega na podzieleniu każdego histogramu przez jego największy element.
5. Na podstawie histogramów skumulowanych należy stworzyć przekształcenie LUT.
Należy tak przeskalować $C_1$ i $C_2$, aby uzyskać jednorodne przekształcenie.
Tablicę $C_1$ wystarczy pomnożyć przez próg podziału.
Tablicę $C_2$ należy przeskalować do przedziału: $<lm+1; 255>$, gdzie $lm$ jest progiem podziału.<br>
$C_{1n} = (lm)*C1;$<br>
$C_{2n} = lm+1 + (255-lm-1)*C2;$<br>
Następnie dwie części tablicy przekodowań należy połączyć.
6. Ostatecznie należy wykonać operację LUT i wyświetlić wynik wyrównywania histogramu.
Porównaj wynik operacji BBHE lub DSIHE z klasycznym HE.

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/'
fileName = 'jet.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

In [ ]:
jet = cv2.imread('jet.bmp', cv2.IMREAD_GRAYSCALE)
fig, ax = plt.subplots(1,2)
fig.set_size_inches(20,10)

ax[0].imshow(jet, 'gray')
histogram = cv2.calcHist([jet], [0], None,  [256], [0, 256])
histogram_cumsum = histogram.cumsum()
histogram_cumsum_scale = (histogram_cumsum/(np.max(histogram_cumsum)/np.max(histogram)))
ax[1].plot(histogram)
ax[1].plot(histogram_cumsum_scale)
plt.show()

In [ ]:
 # dla BBHE
bbhe_lm = int(np.mean(jet))

 # dla DSIHE
polowa = jet.size // 2
polowa_histogram_cumsum = np.abs(histogram_cumsum - polowa)
dsihe_lm = np.argmin(polowa_histogram_cumsum)

def print_image(img, lm, name, histogram):
  H1 = histogram[:lm+1]
  H2 = histogram[lm+1:]

  C1 = np.cumsum(H1)
  C2 = np.cumsum(H2)

  fig, ax = plt.subplots(1,3)
  fig.set_size_inches(35,10)
  fig.suptitle(name, size=50)
  ax[0].plot((H1/np.max(H1))*255)
  ax[0].plot((C1/np.max(C1))*255)
  ax[0].set_title('Wykres H1 i C1', size=25)
  ax[1].plot((H2/np.max(H2))*255)
  ax[1].plot((C2/np.max(C2))*255)
  ax[1].set_title('Wykres H2 i C2', size=25)

  C1n = (C1 / np.max(C1)) * lm
  C2n = (C2 / np.max(C2)) * (255 - lm - 1) + lm + 1

  LUT = np.zeros((256,), dtype=np.uint8)
  LUT[:lm+1] = C1n
  LUT[lm+1:] = C2n

  res = cv2.LUT(img, LUT)

  ax[2].imshow(res, 'gray')
  ax[2].set_title(f"Obraz {name}", size=25)
  plt.show()
print_image(jet, bbhe_lm, 'BBHE', histogram)
print_image(jet, dsihe_lm, 'DSIHE', histogram)

jet_EH = cv2.equalizeHist(jet)
plt.imshow(jet_EH, 'gray')
plt.suptitle('HE', size=25)
plt.show()